In [1]:
# ①  Run once: installs Miniconda under /usr/local and restarts the kernel
!pip install -q condacolab
import condacolab; condacolab.install()


⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


In [2]:
%%bash
# ②  Create env named "sg_env" with the requested tool-chain
conda create -y -n sg_env \
    python=3.7 \
    cudatoolkit=11.0 \
    numpy=1.19       # any >=1.14 works; 1.19 is last to ship for Py3.7
source activate sg_env

# ③  Install PyTorch 1.7.1 + CUDA 11.0 wheels
pip install --no-cache-dir \
    torch==1.7.1+cu110 torchvision==0.8.2+cu110 \
    -f https://download.pytorch.org/whl/torch_stable.html

# ④  Remaining Python deps from the README
pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/sg_env

  added / updated specs:
    - cudatoolkit=11.0
    - numpy=1.19
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.4.26  |       hbd8a1cb_0         149 KB  conda-forge
    cudatoolkit-11.0.3         |      h7761cd4_13       699.0 MB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 KB  conda-forge
    libblas-3.9.0              |31_h59b9bed_openblas          16 KB  conda-forge
    libcblas-3.9.0             |31_he106b2a_openblas          16 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-15.1.0              |       h767d61c_2         810 KB  conda-forge
    libgcc-ng-15.1.0           |       h69a702a_2          34 KB  c



==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda




In [1]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git


fatal: destination path 'stylegan2-ada-pytorch' already exists and is not an empty directory.


In [ ]:
%cd /content/stylegan2-ada-pytorch

In [2]:
!pip install click


  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)


In [3]:
!conda run -n sg_env pip install numpy click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3 psutil scipy pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 29.9 MB/s eta 0:00:00



In [4]:
import os, glob, unicodedata, subprocess
from PIL import Image
import subprocess, os, glob, shutil, re
from google.colab import drive

drive.mount('/content/drive', force_remount=False)
data_root = "/content/drive/MyDrive/Dataset"        # ← 原始画家文件夹
proc_root = "/content/preprocessed"                 # 临时裁剪图
zip_root  = "/content/zips"                         # .zip 供 StyleGAN 用
os.makedirs(proc_root, exist_ok=True)
os.makedirs(zip_root,  exist_ok=True)

def ascii_name(name):
    return unicodedata.normalize("NFKD", name).encode("ascii","ignore").decode() or "artist"

def square_resize(src_dir, size=256):
    artist = ascii_name(os.path.basename(src_dir))
    dst_dir = f"{proc_root}/{artist}"
    if os.path.exists(dst_dir):
        return dst_dir
    os.makedirs(dst_dir, exist_ok=True)

    for p in glob.glob(f"{src_dir}/**/*", recursive=True):
        if p.lower().endswith((".jpg", ".jpeg", ".png")):
            try:
                img = Image.open(p).convert("RGB")
                w, h = img.size
                m = min(w, h)
                left, top = (w-m)//2, (h-m)//2
                img = img.crop((left, top, left+m, top+m)).resize((size, size), Image.LANCZOS)
                img.save(f"{dst_dir}/{os.path.basename(p)}")
            except Exception:
                pass          # 跳过坏图
    return dst_dir

def make_zip(sq_dir):
    artist = ascii_name(os.path.basename(sq_dir))
    zip_path = f"{zip_root}/{artist}_256.zip"
    if os.path.exists(zip_path):
        return zip_path

    print(f"🛠️  Running dataset_tool.py for: {artist}")
    result = subprocess.run([
        "conda", "run", "-n", "sg_env", "python",
        "/content/stylegan2-ada-pytorch/dataset_tool.py",
        "--source", sq_dir,
        "--dest", zip_path
    ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    if result.returncode != 0:
        print(f"❌ Failed on {artist}:")
        print(result.stdout)
        raise RuntimeError("dataset_tool.py failed")

    return zip_path


zip_paths = []
for folder in sorted(glob.glob(f"{data_root}/*")):
    if not os.path.isdir(folder):
        continue
    sq_dir   = square_resize(folder)     # 中心裁剪 → 256²
    zip_path = make_zip(sq_dir)          # 打成 .zip
    zip_paths.append((zip_path, ascii_name(os.path.basename(folder))))

print("✅ 预处理完成，zip 数量:", len(zip_paths))


Mounted at /content/drive
🛠️  Running dataset_tool.py for: Ai_Xuan_
🛠️  Running dataset_tool.py for: Chang_Dai_Chien_
🛠️  Running dataset_tool.py for: Chu_Chu_
🛠️  Running dataset_tool.py for: Giuseppe_Castiglione_
🛠️  Running dataset_tool.py for: Gu_Wenda_
🛠️  Running dataset_tool.py for: He_Baili_
🛠️  Running dataset_tool.py for: He_Jilan_
🛠️  Running dataset_tool.py for: Jin_Ting_Biao_
🛠️  Running dataset_tool.py for: Li_Fang_Ying_
🛠️  Running dataset_tool.py for: Li_Zijian_
🛠️  Running dataset_tool.py for: Ling_Jian_
🛠️  Running dataset_tool.py for: Liu_Kuo_Sung_
🛠️  Running dataset_tool.py for: Liu_Yuan_Shou_
🛠️  Running dataset_tool.py for: Lu_Xin_Jian_
🛠️  Running dataset_tool.py for: Luo_Jian_Wu_
🛠️  Running dataset_tool.py for: Luo_Zhong_Li_
🛠️  Running dataset_tool.py for: Qin_Feng_
🛠️  Running dataset_tool.py for: Qiu_Ying_
🛠️  Running dataset_tool.py for: Wai_Ming
🛠️  Running dataset_tool.py for: Wang_Hui_
🛠️  Running dataset_tool.py for: Yao_Wen_Han_
🛠️  Running dataset_to

In [5]:
!python dataset_tool.py --source /content/preprocessed/Ai_Xuan_ --dest /content/zips/Ai_Xuan__256.zip


python: can't open file '/content/dataset_tool.py': [Errno 2] No such file or directory


In [6]:
!conda run -n sg_env pip install psutil scipy

In [7]:
import shutil
shutil.rmtree("/content/runs", ignore_errors=True)


In [8]:
!wget https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-ffhq-256x256.pkl


--2025-05-09 19:07:44--  https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-ffhq-256x256.pkl
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 44.239.85.224, 54.203.101.202
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|44.239.85.224|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://xfiles.ngc.nvidia.com/org/nvidia/team/research/models/stylegan2/versions/1/files/stylegan2-ffhq-256x256.pkl?ssec-algo=AES256&ssec-key=vqMK8aK39TYOZoySxQEPE%2F5Mwin%2FWBAgXHBY3q4ZYGErk1kxAYbgHMpdJG7PnGQ33HvHny7ek6C3b3%2F6jG11X%2BIodPLu3NVuU5VBxKyoYtJEyKr5E5JU12MDkAtzF%2Bz3Yq67dqR16o68hFaiAy1rmDkurzNsEc4kje9aKntS5nCYuZDqoItQAiPXvaj9dCFVl5I6sQQ4z34rbVLJcCKdL0q5C0f3FMdppBcPBoyDjLHdCl45Tyw%2B%2Fi%2BZ9oPJGWbwMN5fX88Q%2Fuwt6FDrf%2FlCkUzAp6X2zUhK9mQo2xJy0y2nY8oEO4cbTQ52RMODgRAcUxIRmfBjjxnqi2Pw5p19JXEF%2FzLTOpqYTztDROSNGMvdr0uexiTcerzB7j9Ikdy1WkA9sprUVPMrlAAsogv%2BEX285xrDmoswJ1aMnmRbp0AaRdMQwfPRRkslTjSjEby3vszDatB3gg32js5p1l9k85r4

In [43]:
import subprocess, os

os.makedirs("/content/runs", exist_ok=True)

for idx, zip_path in enumerate(zip_paths, 1):
    zip_path = zip_path[0] if isinstance(zip_path, (tuple, list)) else zip_path
    run_dir  = f"/content/runs/run{idx:02d}"

    cmd = [
        "conda", "run", "-n", "sg_env", "python",
        "/content/stylegan2-ada-pytorch/train.py",
        "--data",  zip_path,
        "--outdir", run_dir,
        "--gpus",  "1",
        "--batch", "12",
        "--cfg",   "auto",
        "--snap",  "1",
        "--kimg",  "100",
        "--mirror", "1",
        "--metrics", "none",
    ]

    print(f"\n🚀 Launching {os.path.basename(zip_path)}")
    proc = subprocess.run(cmd, text=True,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.STDOUT)

    if proc.returncode != 0:
        print(f"\n‼️ Training failed on {zip_path}\n{proc.stdout}\n")
        continue

    print(f"✅ Finished {os.path.basename(zip_path)}\n")



🚀 Launching Ai_Xuan__256.zip
✅ Finished Ai_Xuan__256.zip


🚀 Launching Chang_Dai_Chien__256.zip
✅ Finished Chang_Dai_Chien__256.zip


🚀 Launching Chu_Chu__256.zip
✅ Finished Chu_Chu__256.zip


🚀 Launching Giuseppe_Castiglione__256.zip
✅ Finished Giuseppe_Castiglione__256.zip


🚀 Launching Gu_Wenda__256.zip
✅ Finished Gu_Wenda__256.zip


🚀 Launching He_Baili__256.zip
✅ Finished He_Baili__256.zip


🚀 Launching He_Jilan__256.zip
✅ Finished He_Jilan__256.zip


🚀 Launching Jin_Ting_Biao__256.zip
✅ Finished Jin_Ting_Biao__256.zip


🚀 Launching Li_Fang_Ying__256.zip
✅ Finished Li_Fang_Ying__256.zip


🚀 Launching Li_Zijian__256.zip
✅ Finished Li_Zijian__256.zip


🚀 Launching Ling_Jian__256.zip
✅ Finished Ling_Jian__256.zip


🚀 Launching Liu_Kuo_Sung__256.zip
✅ Finished Liu_Kuo_Sung__256.zip


🚀 Launching Liu_Yuan_Shou__256.zip
✅ Finished Liu_Yuan_Shou__256.zip


🚀 Launching Lu_Xin_Jian__256.zip
✅ Finished Lu_Xin_Jian__256.zip


🚀 Launching Luo_Jian_Wu__256.zip
✅ Finished Luo_Jian_Wu__256.zip


In [44]:
# ----------------- Fixed: generate using correct snapshot paths -----------------


gen_root = "/content/gen"
os.makedirs(gen_root, exist_ok=True)

drive_dir = "/content/drive/MyDrive/stylegan_generated"
os.makedirs(drive_dir, exist_ok=True)

run_base = "/content/runs"
run_folders = sorted(glob.glob(os.path.join(run_base, "run*/0000*-*")))  # ✅ 进入真正的训练子目录

for run_path in run_folders:
    run_name = os.path.basename(run_path)

    # 提取艺术家名字
    match = re.match(r"\d{5}-(.+?)(?:__\d+)?-mirror.*", run_name)
    artist_name = match.group(1).replace(" ", "_") if match else run_name

    # 找 snapshot
    pkl_files = sorted(glob.glob(os.path.join(run_path, 'network-snapshot-*.pkl')))
    if not pkl_files:
        print(f"❌ No snapshot found in {run_path}")
        continue
    latest_pkl = pkl_files[-1]

    # 生成输出路径
    gen_dir = os.path.join(gen_root, artist_name)
    os.makedirs(gen_dir, exist_ok=True)

    # 生成图片
    gen_cmd = [
        "conda", "run", "-n", "sg_env", "python",
        "/content/stylegan2-ada-pytorch/generate.py",
        "--outdir", gen_dir,
        "--trunc", "1",
        "--seeds", ",".join(str(i) for i in range(300)),
        "--network", latest_pkl
    ]
    print(f"🖼️ Generating for {artist_name} from {run_name}")
    gen_proc = subprocess.run(gen_cmd, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    if gen_proc.returncode != 0:
        print(f"‼️ Generation failed for {artist_name}\n{gen_proc.stdout}")
        continue
    print("✅ Generation done.")

    # 拷贝到 Google Drive
    drive_subdir = os.path.join(drive_dir, artist_name)
    if os.path.exists(drive_subdir):
        shutil.rmtree(drive_subdir)
    shutil.copytree(gen_dir, drive_subdir)
    print(f"📂 Copied to Drive: {drive_subdir}\n")

print("🎉 All image generations completed and organized by artist in Google Drive!")


🖼️ Generating for Ai_Xuan from 00000-Ai_Xuan__256-mirror-auto1-kimg8-batch12
✅ Generation done.
📂 Copied to Drive: /content/drive/MyDrive/stylegan_generated/Ai_Xuan

🖼️ Generating for Chang_Dai_Chien from 00000-Chang_Dai_Chien__256-mirror-auto1-kimg8-batch12
✅ Generation done.
📂 Copied to Drive: /content/drive/MyDrive/stylegan_generated/Chang_Dai_Chien

🖼️ Generating for Chu_Chu from 00000-Chu_Chu__256-mirror-auto1-kimg8-batch12
✅ Generation done.
📂 Copied to Drive: /content/drive/MyDrive/stylegan_generated/Chu_Chu

🖼️ Generating for Giuseppe_Castiglione from 00000-Giuseppe_Castiglione__256-mirror-auto1-kimg8-batch12
✅ Generation done.
📂 Copied to Drive: /content/drive/MyDrive/stylegan_generated/Giuseppe_Castiglione

🖼️ Generating for Gu_Wenda from 00000-Gu_Wenda__256-mirror-auto1-kimg8-batch12
✅ Generation done.
📂 Copied to Drive: /content/drive/MyDrive/stylegan_generated/Gu_Wenda

🖼️ Generating for He_Baili from 00000-He_Baili__256-mirror-auto1-kimg8-batch12
✅ Generation done.
📂 Copi

In [45]:
import shutil
shutil.rmtree("/content/runs", ignore_errors=True)


In [ ]:
import subprocess, os

os.makedirs("/content/runs", exist_ok=True)

for idx, zip_path in enumerate(zip_paths, 1):
    zip_path = zip_path[0] if isinstance(zip_path, (tuple, list)) else zip_path
    run_dir  = f"/content/runs/run{idx:02d}"

    cmd = [
        "conda", "run", "-n", "sg_env", "python",
        "/content/stylegan2-ada-pytorch/train.py",
        "--data",  zip_path,
        "--outdir", run_dir,
        "--gpus",  "1",
        "--batch", "12",
        "--cfg",   "auto",
        "--snap",  "10",
        "--kimg",  "100",
        "--mirror", "1",
        "--metrics", "none",
        "--augpipe", "bgc"
    ]

    print(f"\n🚀 Launching {os.path.basename(zip_path)}")
    proc = subprocess.run(cmd, text=True,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.STDOUT)

    if proc.returncode != 0:
        print(f"\n‼️ Training failed on {zip_path}\n{proc.stdout}\n")
        continue
    else:
      print(f"✅ Finished {os.path.basename(zip_path)}\n")
    break



🚀 Launching Ai_Xuan__256.zip


In [ ]:
# ----------------- Fixed: generate using correct snapshot paths -----------------


gen_root = "/content/gen"
os.makedirs(gen_root, exist_ok=True)

drive_dir = "/content/drive/MyDrive/stylegan_generated"
os.makedirs(drive_dir, exist_ok=True)

run_base = "/content/runs"
run_folders = sorted(glob.glob(os.path.join(run_base, "run*/0000*-*")))  # ✅ 进入真正的训练子目录

for run_path in run_folders:
    run_name = os.path.basename(run_path)

    # 提取艺术家名字
    match = re.match(r"\d{5}-(.+?)(?:__\d+)?-mirror.*", run_name)
    artist_name = match.group(1).replace(" ", "_") if match else run_name

    # 找 snapshot
    pkl_files = sorted(glob.glob(os.path.join(run_path, 'network-snapshot-*.pkl')))
    if not pkl_files:
        print(f"❌ No snapshot found in {run_path}")
        continue
    latest_pkl = pkl_files[-1]

    # 生成输出路径
    gen_dir = os.path.join(gen_root, artist_name)
    os.makedirs(gen_dir, exist_ok=True)

    # 生成图片
    gen_cmd = [
        "conda", "run", "-n", "sg_env", "python",
        "/content/stylegan2-ada-pytorch/generate.py",
        "--outdir", gen_dir,
        "--trunc", "1",
        "--seeds", ",".join(str(i) for i in range(300)),
        "--network", latest_pkl
    ]
    print(f"🖼️ Generating for {artist_name} from {run_name}")
    gen_proc = subprocess.run(gen_cmd, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    if gen_proc.returncode != 0:
        print(f"‼️ Generation failed for {artist_name}\n{gen_proc.stdout}")
        continue
    print("✅ Generation done.")

    # 拷贝到 Google Drive
    drive_subdir = os.path.join(drive_dir, artist_name)
    if os.path.exists(drive_subdir):
        shutil.rmtree(drive_subdir)
    shutil.copytree(gen_dir, drive_subdir)
    print(f"📂 Copied to Drive: {drive_subdir}\n")

print("🎉 All image generations completed and organized by artist in Google Drive!")
